In [1]:
import sys
import logging
import os
import json
import pandas as pd

logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(message)s")

# LOAD DATA

In [2]:
DATA_FOLDER = os.getcwd() + '/review_scorer/data/'
SENTI_PATH = DATA_FOLDER + 'SentiWord_info.json'
DATA_PATH = DATA_FOLDER + 'data_origin.csv'
data: pd.DataFrame

with open(SENTI_PATH, mode='rt', encoding='UTF8') as f:
    senti = pd.DataFrame.from_dict(json.load(f))

data = pd.read_csv(DATA_PATH, encoding='UTF8')
data = data.dropna(axis=0)
data = data.sample(frac=1).reset_index(drop=True)

# TOKENIZER

In [3]:
from tqdm import tqdm
from twkorean import TwitterKoreanProcessor

processor = TwitterKoreanProcessor()
tokenize = processor.tokenize_to_strings
tokens = [tokenize(_) for _ in tqdm(data.review)]

100%|████████████████████████████████████| 71904/71904 [08:36<00:00, 139.25it/s]


# SET REVIEW SCORER

In [4]:
from review_scorer import ReviewScorer

# Review scorer needs tokens of datas to train when initializing it.
# 리뷰 채점기 클래스를 생성할 때, 토크나이즈 된 데이터를 인자로 주어야 합니다.
rs = ReviewScorer(sentences=tokens, senti_dict_path=SENTI_PATH)

collecting all words and their counts
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
PROGRESS: at sentence #10000, processed 1101910 words, keeping 28163 word types
PROGRESS: at sentence #20000, processed 2200031 words, keeping 37331 word types
PROGRESS: at sentence #30000, processed 3296263 words, keeping 43244 word types
PROGRESS: at sentence #40000, processed 4396701 words, keeping 47564 word types
PROGRESS: at sentence #50000, processed 5475354 words, keeping 49702 word types
PROGRESS: at sentence #60000, processed 6538131 words, keeping 50953 word types
PROGRESS: at sentence #70000, processed 7627974 words, keeping 51655 word types
collected 51667 word types from a corpus of 7833944 raw words and 71904 sentences
Creating a fresh vocabulary
ReviewScorer lifecycle event {'msg': 'effective_min_count=5 retains 22483 unique words (43.52% of original 51667, drops 29184)', 'datetime': '2022-08-10T14:20:36.775294', 'gensim': '4.2.0', 'python': '3.7.12 | packaged by cond

## TAGGING RIVIEW SCORER's SENTIMENTAL DICTIONARY

In [5]:
# Tagging review scorer's sentimental dictionary by category.
# 카테고리에 따라 리뷰 채점기의 감성사전을 태깅합니다.
rs.tag(categories={'taste': ['맛', '맛있다', '맛없다'],
                   'price': ['가격', '싸다', '비싸다', '저렴'],
                   'service': ['서비스', '친절', '싸가지'],
                   'atmosphere': ['인테리어', '분위기']}, topn=1000)

taste tagged
price tagged
service tagged
atmosphere tagged


# SCORING WITH REVIEW SCORER

In [6]:
start = 50000
for i in range(start, start + 10):
    print('index: ', i)
    print(rs.score_review(tokenize(data.review.iloc[i])))
    print(data.review.iloc[i])
    print()

index:  50000
{'taste': 7, 'price': 4, 'service': 3, 'atmosphere': 2}
#을지로 #입정동 #을지면옥"맛의 대향연이였던 냉면 한 그릇"1. 을지면옥을 방문한지 3년이 훌쩍 넘었다. 2017년 당시 너무 짜진 육수에 실망을 했으나 술안주로 먹기에는 적절한 간이라 나쁘지는 않았던 것 같다.영하 10도가 넘는 칼바람이 불던 날, 을지로길을 걷다 발견한 을지면옥의 올드스쿨 바이브 가득한 간판에 이끌려 예전과 많이 달라졌나 궁금해 냉면 한 그릇 청했다.2. 의정부 평양면옥의 특징인 고추가루가 솔솔 올라간 냉면 한 그릇이 서빙이 됐는데, 지금까지는 몰랐던 냉면 그릇의 사이즈에 깜짝 놀랐다.생각보다 큰데다 양도 상당하다. 보통 냉면 먹을 때 육수 먼저 원샷으로 끝내고 추가 육수 받아 다시 시작하는데 이집은 그럴 필요가 없을 정도로 육수와 국수의 양이 넉넉하다.을밀대 본점에서 "양많이"로 주문하면 나오는 그 정도의 양이 기본으로 서빙이 된다.가격이 12,000원이라 왠지 그만큼은 먹어야 본전이 될 것 같은 느낌이 들기도 하지만 ㅎㅎㅎ3. 맑은 국물과는 대조적으로 육향이 굉장이 진하고 간도 간간하다. 예전보다는 좀 더 순화된 간이지만 결코 전에 비해 덜짜졌다고 말하긴 어렵다.그런데 맛이 폭발한다. 먼저 육수는 육향도 진하고 간도 있지만 감칠맛도 강하다. 거부감 스러운 감칠맛은 아니지만 차가운 곰탕을 먹는 듯한 맛이다.이집 메뉴에 소고기국밥이 있는데, 요즘 여기 육수 스타일을 보면 맛있을 것 같다.4. 면을 살살 풀어서 육수에 곡기를 주고 마시면 또다른 맛이다. 감칠맛이 순화되서 부드럽고 고급스러운 맛으로 변한다.면은 겨울이라서 그런지 메밀향이 꽤 좋다. 감칠맛 육수가 묻어 있는 메밀향 좋은 면빨은 행복 그 자체다.그런데 여기서 끝나지 않고 가끔 톡톡 씹히는 참깨의 고소함이 일품이다. 필동면옥도 참깨를 좀 올리지만 을지면옥의 참깨 양이 압도적으로 많아 참깨의 고소함을 씹을 확률이 더 높다.더불어 씹히는 고추가루의 알싸한 맛까지